<a href="https://colab.research.google.com/github/nyk3151/PHTrans_nyk_test/blob/main/PHTrans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PHTrans**

This is experimental development of PHTrans for research.


In [1]:
# mount to Google colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import

In [14]:
# PHTransのリポジトリをクローン
!git clone https://github.com/lseventeen/PHTrans.git /content/PHTrans

# 必要なライブラリをインストール
!pip install batchgenerators

# PHTransリポジトリ内のnnU-Net (v1) をインストール
!cd /content/PHTrans/nnUNet && pip install -e .

# PHTrans本体をインストール
!cd /content/PHTrans/PHTrans && pip install -e .

print("✅ Installation complete.")

fatal: destination path '/content/PHTrans' already exists and is not an empty directory.
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


Obtaining file:///content/PHTrans/nnUNet
  Preparing metadata (setup.py) ... done
  Using cached dicom2nifti-2.6.2-py3-none-any.whl.metadata (1.5 kB)
  Using cached medpy-0.5.2.tar.gz (156 kB)
ERROR: Operation cancelled by user
^C
^C
✅ Installation complete.


## Preprocessing

In [5]:
!pip show nnunet

In [1]:
import os

# 各フォルダへのパスを定義
raw_data_base = '/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUNet_raw_data_base'
preprocessed_folder = '/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUNet_preprocessed'
results_folder = '/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUNet_results'

# フォルダが存在しない場合は作成
os.makedirs(preprocessed_folder, exist_ok=True)
os.makedirs(results_folder, exist_ok=True)

# 3つの必須環境変数を設定
%env nnUNet_raw_data_base={raw_data_base}
%env nnUNet_preprocessed={preprocessed_folder}
%env RESULTS_FOLDER={results_folder}

print("✅ Environment variables set.")

env: nnUNet_raw_data_base=/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUNet_raw_data_base
env: nnUNet_preprocessed=/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUNet_preprocessed
env: RESULTS_FOLDER=/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUNet_results
✅ Environment variables set.


In [ ]:
import os
import SimpleITK as sitk

In [ ]:
# 画像とラベルデータのディレクトリ
image_dir = '/content/drive/MyDrive/Research/Deep_learning/Disec_net/images'
label_dir = '/content/drive/MyDrive/Research/Deep_learning/Disec_net/labels'

# プロパティ比較関数
def compare_image_properties(image1, image2):
    properties = ['Size', 'Spacing', 'Origin', 'Direction']
    match = True

    for prop in properties:
        if prop == 'Size':
            val1 = image1.GetSize()
            val2 = image2.GetSize()
        elif prop == 'Spacing':
            val1 = image1.GetSpacing()
            val2 = image2.GetSpacing()
        elif prop == 'Origin':
            val1 = image1.GetOrigin()
            val2 = image2.GetOrigin()
        elif prop == 'Direction':
            val1 = image1.GetDirection()
            val2 = image2.GetDirection()

        print(f"{prop}:")
        print(f"  Image 1: {val1}")
        print(f"  Image 2: {val2}")
        if val1 == val2:
            print(f"  -> {prop}が一致しています。\n")
        else:
            print(f"  -> {prop}が一致していません。\n")
            match = False

    if match:
        print("全てのプロパティが一致しています。")
    else:
        print("一部のプロパティが一致していません。")

# 画像とラベルを一括で処理する関数
def process_images_and_labels(image_dir, label_dir):
    image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.nii.gz')])
    label_files = sorted([f for f in os.listdir(label_dir) if f.endswith('.nii.gz')])

    for image_file, label_file in zip(image_files, label_files):
        image_path = os.path.join(image_dir, image_file)
        label_path = os.path.join(label_dir, label_file)

        print(f"Processing {image_file} and {label_file} for comparison...")

        # 画像とラベルの読み込み
        image = sitk.ReadImage(image_path)
        resampled_segmentation = sitk.ReadImage(label_path)

        # プロパティを比較
        compare_image_properties(image, resampled_segmentation)

# 画像とラベルのプロパティを一括で比較
process_images_and_labels(image_dir, label_dir)


Processing case_0001_0000.nii.gz and case_0001.nii.gz for comparison...
Size:
  Image 1: (512, 512, 549)
  Image 2: (512, 512, 549)
  -> Sizeが一致しています。

Spacing:
  Image 1: (0.7421879768371582, 0.7421879768371582, 1.25)
  Image 2: (0.7421879768371582, 0.7421879768371582, 1.25)
  -> Spacingが一致しています。

Origin:
  Image 1: (-190.0, -190.0, -605.0)
  Image 2: (-190.0, -190.0, -605.0)
  -> Originが一致しています。

Direction:
  Image 1: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
  Image 2: (1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
  -> Directionが一致しています。

全てのプロパティが一致しています。
Processing case_0002_0000.nii.gz and case_0002.nii.gz for comparison...
Size:
  Image 1: (512, 512, 831)
  Image 2: (512, 512, 831)
  -> Sizeが一致しています。

Spacing:
  Image 1: (0.7229999899864197, 0.7229999899864197, 1.0)
  Image 2: (0.7229999899864197, 0.7229999899864197, 1.0)
  -> Spacingが一致しています。

Origin:
  Image 1: (-192.60699462890625, -184.7946014404297, -1518.5)
  Image 2: (-192.60699462890625, -184.7946014404297, -1518.5)


In [2]:
image_dir = '/content/drive/MyDrive/Research/Deep_learning/Disec_net/images'
label_dir = '/content/drive/MyDrive/Research/Deep_learning/Disec_net/labels'

In [3]:
import os
import shutil
from sklearn.model_selection import train_test_split

# データリストの取得
image_files = sorted([f for f in os.listdir(image_dir) if f.endswith('.nii.gz')])
label_files = sorted([f for f in os.listdir(label_dir) if f.endswith('.nii.gz')])

len(image_files)

62

In [4]:
total_samples = len(image_files)
print(f"Total samples: {total_samples}")

Total samples: 62


In [5]:
from sklearn.model_selection import train_test_split

# トレーニング、検証、テストデータの分割サイズ
train_size = 60 / total_samples
test__size_ratio = 2/ total_samples


# トレーニングデータと検証データに分割
train_images, test_images, train_labels, test_labels = train_test_split(
    image_files, label_files, train_size=train_size, random_state=42
)


In [6]:
#outputディレクトリのパス
output_dir = '/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUNet_raw_data_base'

In [7]:
import os
# ディレクトリの作成
os.makedirs(os.path.join(output_dir, 'nnUNet_raw_data', 'Task001_DSeg', 'imagesTr'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'nnUNet_raw_data', 'Task001_DSeg', 'labelsTr'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'nnUNet_raw_data', 'Task001_DSeg', 'imagesTs'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'nnUNet_raw_data', 'Task001_DSeg', 'labelsTs'), exist_ok=True)

In [8]:
# トレーニングデータのコピー
for img, lbl in zip(train_images, train_labels):
    shutil.copy(os.path.join(image_dir, img), os.path.join(output_dir, 'nnUNet_raw_data', 'Task001_DSeg', 'imagesTr'))
    shutil.copy(os.path.join(label_dir, lbl), os.path.join(output_dir, 'nnUNet_raw_data', 'Task001_DSeg', 'labelsTr'))


# テストデータのコピー
for img, lbl in zip(test_images, test_labels):
    shutil.copy(os.path.join(image_dir, img), os.path.join(output_dir, 'nnUNet_raw_data', 'Task001_DSeg', 'imagesTs'))
    shutil.copy(os.path.join(label_dir, lbl), os.path.join(output_dir, 'nnUNet_raw_data', 'Task001_DSeg', 'labelsTs'))

In [10]:
import os
import json

# --- パス設定 ---
task_folder = '/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_data_base/nnUNet_raw_data/Task001_DSeg'
images_tr_folder = os.path.join(task_folder, 'imagesTr')
images_ts_folder = os.path.join(task_folder, 'imagesTs')

# --- ファイルリスト取得 ---
train_images = sorted(os.listdir(images_tr_folder))
test_images = sorted(os.listdir(images_ts_folder))

# --- dataset.json作成 ---
print("\n--- Creating the final version of dataset.json ---")
dataset_json = {
    "name": "DSegmentation",
    "description": "Segmentation of aortic dissection",
    "modality": { "0": "CT" },
    "labels": {
        "0": "background", "1": "True_Lumen", "2": "False_Lumen", "3": "Thrombosis"
    },
    "numTraining": len(train_images),
    "numTest": len(test_images),
    "training": [],
    "test": []
}

# 🚨 このロジックが最終版です
for image_file in train_images:
    # labelファイル名は、imageファイル名から "_0000" を取り除いたもの
    label_file = image_file.replace('_0000.nii.gz', '.nii.gz')

    dataset_json["training"].append({
        "image": f"./imagesTr/{image_file}",
        "label": f"./labelsTr/{label_file}"
    })

for image_file in test_images:
    dataset_json["test"].append(f"./imagesTs/{image_file}")

# --- ファイルに保存 ---
json_path = os.path.join(task_folder, 'dataset.json')
with open(json_path, 'w') as f:
    json.dump(dataset_json, f, indent=4)

print(f"\n✅ Successfully created and saved the final dataset.json.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_data_base/nnUNet_raw_data/Task001_DSeg/imagesTr'

In [7]:
import json

# 確認したいdataset.jsonのフルパス
json_path = '/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUNet_raw_database/nnUNet_raw_data/Task001_DSeg/dataset.json'

print(f"--- Checking contents of: {json_path} ---")

try:
    with open(json_path, 'r') as f:
        data = json.load(f)
        # trainingの最初の5件を表示して確認します
        print("First 5 entries in 'training':")
        print(json.dumps(data['training'][:5], indent=4))
except FileNotFoundError:
    print("ERROR: File not found at the specified path.")
except Exception as e:
    print(f"An error occurred: {e}")

--- Checking contents of: /content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUNet_raw_database/nnUNet_raw_data/Task001_DSeg/dataset.json ---
ERROR: File not found at the specified path.


In [ ]:
# Colab/Jupyterで環境変数を設定する正しい方法
%env nnUNet_raw_data_base=/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database
%env nnUNet_preprocessed=/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_preprocessed
%env RESULTS_FOLDER=/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_results

env: nnUNet_raw_data_base=/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database
env: nnUNet_preprocessed=/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_preprocessed
env: RESULTS_FOLDER=/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_results


In [ ]:
#!mv /content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database/dataset.json /content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database/nnUNet_raw_data/Task001_DSeg/

In [ ]:
!ls /content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database/nnUNet_raw_data/Task001_DSeg/labelsTr | head

case_0001.nii.gz
case_0002.nii.gz
case_0003.nii.gz
case_0004.nii.gz
case_0005.nii.gz
case_0006.nii.gz
case_0007.nii.gz
case_0008.nii.gz
case_0009.nii.gz
case_0010.nii.gz


In [12]:
!pip show nnunet

In [11]:
!nnUNet_plan_and_preprocess -t 001 --verify_dataset_integrity

/bin/bash: line 1: nnUNet_plan_and_preprocess: command not found


In [ ]:
# nnUNet_raw_data_baseに設定したフォルダの中身をすべて表示します
!ls -R /content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database

/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database:
dataset.json  nnUNet_raw_data

/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database/nnUNet_raw_data:
Task001_DSeg

/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database/nnUNet_raw_data/Task001_DSeg:
imagesTr  imagesTs  labelsTr  labelsTs

/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database/nnUNet_raw_data/Task001_DSeg/imagesTr:
case_0001_0000.nii.gz  case_0021_0000.nii.gz  case_0041_0000.nii.gz
case_0002_0000.nii.gz  case_0022_0000.nii.gz  case_0042_0000.nii.gz
case_0003_0000.nii.gz  case_0023_0000.nii.gz  case_0043_0000.nii.gz
case_0004_0000.nii.gz  case_0024_0000.nii.gz  case_0044_0000.nii.gz
case_0005_0000.nii.gz  case_0025_0000.nii.gz  case_0047_0000.nii.gz
case_0006_0000.nii.gz  case_0026_0000.nii.gz  case_0048_0000.nii.gz
case_0007_0000.nii.gz  case_0027_0000.nii.gz  case_0049_0000.nii.gz
case_0008_0000.nii.gz  case_0028_0000.nii.gz  case_0050_

In [ ]:
!ls -R /content/drive/MyDrive/Research/Deep_learning/PHTrans

/content/drive/MyDrive/Research/Deep_learning/PHTrans:
nnUnet_preprocessed  nnUnet_raw_database  nnUnet_results

/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_preprocessed:

/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database:
dataset.json  nnunet_raw_data

/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database/nnunet_raw_data:
Task001_DSeg

/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database/nnunet_raw_data/Task001_DSeg:
imagesTr  imagesTs  labelsTr  labelsTs

/content/drive/MyDrive/Research/Deep_learning/PHTrans/nnUnet_raw_database/nnunet_raw_data/Task001_DSeg/imagesTr:
case_0001_0000.nii.gz  case_0021_0000.nii.gz  case_0041_0000.nii.gz
case_0002_0000.nii.gz  case_0022_0000.nii.gz  case_0042_0000.nii.gz
case_0003_0000.nii.gz  case_0023_0000.nii.gz  case_0043_0000.nii.gz
case_0004_0000.nii.gz  case_0024_0000.nii.gz  case_0044_0000.nii.gz
case_0005_0000.nii.gz  case_0025_0000.nii.gz  case_0047_0000.nii.gz
cas

## Training

In [ ]:
!pip install batchgenerators

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
import os

%env nnUNet_raw_data_base=/content/drive/MyDrive/Research/Deep_learning/New_net/datasets/nnunet_data/nnUnet_raw_data
%env nnUNet_preprocessed=/content/drive/MyDrive/Research/Deep_learning/New_net/datasets/nnunet_data/nnUnet_preprocessed
%env RESULTS_FOLDER=/content/drive/MyDrive/Research/Deep_learning/New_net/datasets/nnunet_data/PHTrans_results


# 環境変数の設定
#os.environ['nnUNet_raw'] = '/content/drive/MyDrive/New_net/datasets/nnunet_data/nnUnet_raw_data'
#os.environ['nnUNet_preprocessed'] = '/content/drive/MyDrive/New_net/datasets/nnunet_data/nnUnet_preprocessed'
#os.environ['nnUNet_results'] = '/content/drive/MyDrive/New_net/datasets/nnunet_data/PHTrans_results'

print(f"RESULTS_FOLDER = {os.environ.get('RESULTS_FOLDER')}")
print(f"nnUNet_preprocessed = {os.environ.get('nnUNet_preprocessed')}")
print(f"nnUNet_raw = {os.environ.get('nnUNet_raw')}")


env: nnUNet_raw_data_base=/content/drive/MyDrive/Research/Deep_learning/New_net/datasets/nnunet_data/nnUnet_raw_data
env: nnUNet_preprocessed=/content/drive/MyDrive/Research/Deep_learning/New_net/datasets/nnunet_data/nnUnet_preprocessed
env: RESULTS_FOLDER=/content/drive/MyDrive/Research/Deep_learning/New_net/datasets/nnunet_data/PHTrans_results
RESULTS_FOLDER = /content/drive/MyDrive/Research/Deep_learning/New_net/datasets/nnunet_data/PHTrans_results
nnUNet_preprocessed = /content/drive/MyDrive/Research/Deep_learning/New_net/datasets/nnunet_data/nnUnet_preprocessed
nnUNet_raw = /content/drive/MyDrive/New_net/datasets/nnunet_data/nnUnet_raw_data


In [ ]:
# preprocessedフォルダの中身をリスト表示します
!ls /content/drive/MyDrive/Research/Deep_learning/New_net/datasets/nnunet_data/nnUnet_preprocessed

Dataset001_TLSeg  Dataset002_AOSeg  Dataset003_DSeg  Task004_DSeg


In [ ]:
!ls /content/drive/MyDrive/Research/Deep_learning/New_net/datasets/nnunet_data/nnUnet_preprocessed/Task004_DSeg

dataset_fingerprint.json  nnUNetPlans_2d	  nnUNetPlans.json
dataset.json		  nnUNetPlans_3d_fullres
gt_segmentations	  nnUNetPlans_3d_lowres


In [ ]:
!PHTrans_train -task 004



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

wandb: Tracking run with wandb version 0.21.0
wandb: W&B syncing is set to `offline` in this directory. Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
Traceback (most recent call last):
  File "/usr/local/bin/PHTrans_train", line 33, in <module>
    sys.exit(load_entry_point('phtrans', 'console_scripts', 'PHTrans_train')())
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/PHTrans/PHTrans/phtrans/run/run_training.py", line 123, in main
    task = convert_id_to_task_name(task_id)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-pac

## Learning

##Inferance